# 用Keras实现风格迁移
## 一、导入所需库

In [1]:
from tensorflow import keras
import tensorflow.compat.v2.keras.backend as K
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.applications import vgg19
import tensorflow as tf
# tf.compat.v1.disable_eager_execution()

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.compat.v1.keras.backend import set_session
from tensorflow.compat.v1 import ConfigProto, Session
config = ConfigProto(allow_soft_placement=True)

config.gpu_options.per_process_gpu_memory_fraction = 0.6
set_session(Session(config=config))

## 查看VGG19的模型

In [2]:
conv_base = vgg19.VGG19(include_top=False)
conv_base.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

## 定义初始变量

In [7]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
target_image_path = '/home/oneran/temp.JPEG'    #需要变换的路径
style_referrence_image_path = '/home/oneran/Downloads/referrence_style.jpg'

width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

## 引入辅助函数

In [8]:
def preprocessing_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)           
    return img

def deprocess_img(x):
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

## 加载预训练的VGG19网络，并且将其引用与三张图像

In [9]:
tf.compat.v1.disable_eager_execution()

In [10]:
target_img = K.constant(preprocessing_image(target_image_path))
style_referrence_image = K.constant(preprocessing_image(style_referrence_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3))

input_tensor = K.concatenate([target_img, style_referrence_image, combination_image], axis=0) # 将三张图片连接起来  拼接在第四个轴上

model = vgg19.VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)
print('Model loaded!')

Model loaded!


## 定义内容损失和风格损失

In [11]:
def content_loss(base, combination):
    return K.sum(K.square(combination-base))   #均方和作为内容损失

def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))   #风格损失为矩阵内积
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S-C)) / (4. * (channels ** 2) * (size ** 2))

## 总变差损失

In [12]:
def total_variation_loss(x):
    a = K.square(
        x[:, :img_height-1, :img_width-1, :] - x[:, 1:, img_width-1:, :]            #去掉边缘影响
    )
    b = K.square(
        x[:, :img_height-1, :img_width-1, :] - x[:, img_height-1, 1, :]
    )
    return K.sum(K.pow(a+b, 1.25))

## 定义最终损失

In [13]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
content_layer = 'block5_conv2'
style_layer = [ 'block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(target_image_features, combination_features)

for layer_name in style_layer:
    layer_features = outputs_dict[layer_name]
    style_referrence_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    s1 = style_loss(style_referrence_features, combination_features)
    loss += (style_weight / len(style_layer)) * s1

loss = loss + total_variation_weight * total_variation_loss(combination_image)

## 设置梯度下降过程

In [14]:
grads = K.gradients(loss, combination_image)[0]
fetch_loss_and_grads = K.function([combination_image], [loss, grads])

class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grads_value = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grads_value = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grads_value = grads_value
        return self.loss_value
    
    def grads(self, x):
        assert self.loss_value is not None
        grads_value = np.copy(self.grads_value)
        self.loss_value = None
        self.grads_value = None
        return grads_value

evaluator = Evaluator()

## 风格迁移循环

In [15]:
from scipy.optimize import fmin_l_bfgs_b
from matplotlib.pyplot import imsave
import time

result_prefix = '/home/oneran/testimages/my_results_'
iterations = 100

x = preprocessing_image(target_image_path)
x = x.flatten()
for i in range(iterations):
    print('Start of iterations:', i)
    start_Time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x, fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_img(img)
    fname = result_prefix + 'at_iteration_%d_img.jpg' %i
    imsave(fname, img)
    print('Image saved as', fname)
    end_time = time.time()
    print('Iterations %d completed in %ds' %(i, end_time-start_Time))

Start of iterations: 0
Current loss value: 3219840000.0
Image saved as /home/oneran/testimages/my_results_at_iteration_0_img.jpg
Iterations 0 completed in 19s
Start of iterations: 1
Current loss value: 1528704000.0
Image saved as /home/oneran/testimages/my_results_at_iteration_1_img.jpg
Iterations 1 completed in 11s
Start of iterations: 2
Current loss value: 1045426240.0
Image saved as /home/oneran/testimages/my_results_at_iteration_2_img.jpg
Iterations 2 completed in 11s
Start of iterations: 3
Current loss value: 823283140.0
Image saved as /home/oneran/testimages/my_results_at_iteration_3_img.jpg
Iterations 3 completed in 11s
Start of iterations: 4
Current loss value: 668472800.0
Image saved as /home/oneran/testimages/my_results_at_iteration_4_img.jpg
Iterations 4 completed in 11s
Start of iterations: 5
Current loss value: 580536960.0
Image saved as /home/oneran/testimages/my_results_at_iteration_5_img.jpg
Iterations 5 completed in 11s
Start of iterations: 6
Current loss value: 510830